In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [2]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    if t%10==1:
        test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 1.645947  [   64/60000]
loss: 1.567707  [ 6464/60000]
loss: 1.651063  [12864/60000]
loss: 1.453689  [19264/60000]
loss: 1.487172  [25664/60000]
loss: 1.469282  [32064/60000]
loss: 1.404597  [38464/60000]
loss: 1.512972  [44864/60000]
loss: 1.381101  [51264/60000]
loss: 1.336121  [57664/60000]
Epoch 2
-------------------------------
loss: 1.350729  [   64/60000]
loss: 1.244236  [ 6464/60000]
loss: 1.339114  [12864/60000]
loss: 1.165264  [19264/60000]
loss: 1.183462  [25664/60000]
loss: 1.159586  [32064/60000]
loss: 1.107911  [38464/60000]
loss: 1.235351  [44864/60000]
loss: 1.123422  [51264/60000]
loss: 1.075255  [57664/60000]
Test Error: 
 Accuracy: 79.9%, Avg loss: 1.043530 

Epoch 3
-------------------------------
loss: 1.111046  [   64/60000]
loss: 0.989692  [ 6464/60000]
loss: 1.077824  [12864/60000]
loss: 0.950332  [19264/60000]
loss: 0.960872  [25664/60000]
loss: 0.931401  [32064/60000]
loss: 0.889410  [38464/60000]
loss: 1.023115  [4

In [5]:
torch.save(model.state_dict(), "model.pth")
print(model.state_dict().keys())
print("Saved PyTorch Model State to model.pth")

odict_keys(['linear_relu_stack.0.weight', 'linear_relu_stack.0.bias', 'linear_relu_stack.2.weight', 'linear_relu_stack.2.bias', 'linear_relu_stack.4.weight', 'linear_relu_stack.4.bias'])
Saved PyTorch Model State to model.pth


In [6]:
from json import JSONEncoder
import json

class EncodeTensor(JSONEncoder, torch.utils.data.Dataset):
    def default(self, obj):
        if isinstance(obj, torch.Tensor):
            return obj.cpu().detach().numpy().tolist()
        return super(EncodeTensor, self).default(obj)

with open('torch_weights.json', 'w') as json_file:
    json.dump(model.state_dict(), json_file,cls=EncodeTensor)

In [17]:
with open('weights.bin', 'wb' ) as f: 
  for key, value in model.state_dict().items():
    arr = value.cpu().detach().numpy()
    print(arr.shape, arr.ndim)
    f.write(arr)

(512, 784) 2
(512,) 1
(512, 512) 2
(512,) 1
(10, 512) 2
(10,) 1


In [18]:
for key, value in model.state_dict().items():
  print(value.cpu().detach().numpy())

[[ 0.0279671   0.00938362 -0.02349045 ... -0.01774568  0.03277795
   0.03243467]
 [ 0.01100129  0.01008523  0.03203167 ... -0.01453802  0.03108073
  -0.01675717]
 [-0.00873586 -0.01778367 -0.01929924 ...  0.00893023  0.00272433
  -0.02481144]
 ...
 [-0.01636105  0.01383047 -0.0268059  ...  0.02908236  0.00608143
   0.02359631]
 [ 0.01431939 -0.01392078 -0.0309662  ...  0.00408989  0.03498786
  -0.01710635]
 [ 0.00312846  0.01463996  0.03497315 ... -0.01467601  0.01955105
   0.01244057]]
[ 2.90196319e-03  4.30605225e-02 -1.82794605e-03  3.34332250e-02
  1.70415733e-02  4.75736195e-03  2.93841958e-02  1.67108979e-02
 -6.12115080e-04 -2.55043972e-02  3.96722443e-02  3.22613679e-02
  7.06300559e-03  3.90617400e-02  1.86100474e-03  1.84419174e-02
  3.38026113e-03 -8.95516574e-03 -3.60312080e-03  1.91219803e-02
 -2.32318565e-02 -1.19523648e-02  2.74080113e-02 -7.06782890e-03
  3.11347060e-02  4.03756052e-02  1.49224130e-02  4.29224281e-04
 -9.92678106e-03  5.79616800e-03  7.27736857e-03  9.6